In [166]:
from bs4 import BeautifulSoup #library for web scraping
import requests  # library to handle requests
import json  # library to handle JSON files
import xml
import pandas as pd #Python library data manipulation and analysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

print('Libraries imported!')

Libraries imported!


# Problem 1

In [167]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
source = requests.get(url).text

soup = BeautifulSoup(source, 'lxml')#Beautiful Soup to Parse the url page

table=soup.find('table') #Finds the required table area

#List initialization to collect the Postalcodes, Boroughs and Neighborhoods
postalcode=[]
borough=[]
neighborhood=[]

x=table.tbody #Navigation to the body of the table


In [168]:
for tr in x.find_all('tr'): #Iterating through rows and columns
    td=tr.find_all('td')
    for x in td:
        #Exception handling to bypass the errors
        try:
            if x.span.text!='Not assigned':
                postalcode.append(x.b.text) #Collecting the Postalcodes
        except:
            pass
        try:
            if x.span.text!='Not assigned': 
                my=x.span.get_text(separator=u' ').split(' ')
                
                if my[1]=='York':
                    my_b= my[0]+' '+my[1]
                    borough.append(my_b)
                    my_nei=my[2:]
                    if my_nei==['\n'] or my_nei==[]:
                        neighborhood.append(my_b)
                    else:
                        my_nei1=' '.join(my_nei)
                        neighborhood.append(my_nei1.replace('(',',',5).replace(')',',',5).replace('/',',',5).strip(', '))
                
                elif my[1]=='Toronto':
                    my_b=my[0]+' '+my[1]
                    borough.append(my_b)
                    my_nei=my[2:]
                    if my_nei==['\n'] or my_nei==[]:
                        neighborhood.append(my_b)
                    else:
                        my_nei1=' '.join(my_nei)
                        neighborhood.append(my_nei1.replace('(',',',5).replace(')',',',5).replace('/',',',5).strip(', '))
                
                elif my[1]=='Park':
                    my_b=my[0]+' '+my[1]
                    borough.append(my_b)
                    my_nei=my[2:]
                    if my_nei==['\n'] or my_nei==[]:
                        neighborhood.append(my_b)
                    else:
                        my_nei1=' '.join(my_nei)
                        neighborhood.append(my_nei1.replace('(',',',5).replace(')',',',5).replace('/',',',5).strip(', '))
                
                else:
                    my_b=my[0]
                    borough.append(my_b)
                    my_nei=my[2:]
                    if my_nei==['\n'] or my_nei==[]:
                        neighborhood.append(my_b)
                    else:
                        my_nei1=' '.join(my_nei)
                        neighborhood.append(my_nei1.replace('(',',',5).replace(')',',',5).replace('/',',',5).strip(', '))
       
    
         
        except:
            pass

In [169]:
#Constructing the main dataframe 
main_df=pd.DataFrame({'PostalCode':postalcode,
                 'Borough':borough,
                 'Neighborhood':neighborhood})

In [170]:
#Fixing the order of the columns in the main dataframe
main_df=main_df[['PostalCode','Borough','Neighborhood']]

In [171]:
#Visualizing the first few rows of the dataframe
main_df.head(10)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park , Harbourfront"
3,M6A,North York,"Lawrence Manor , Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Malvern , Rouge"
7,M3B,North York,"Don Mills , North"
8,M4B,East York,"Parkview Hill , Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District , Ryerson"


In [172]:
main_df.shape

(103, 3)

# Problem 2

In [205]:
geo_df = pd.read_csv('https://cocl.us/Geospatial_data')
geo_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [212]:
geo_df.rename(columns = {'Postal Code':'PostalCode'}, inplace = True)
geo_df.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [211]:
geo_merged = pd.merge(main_df, geo_df, on = 'PostalCode')
geo_merged.head(12)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor , Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Malvern , Rouge",43.806686,-79.194353
7,M3B,North York,"Don Mills , North",43.745906,-79.352188
8,M4B,East York,"Parkview Hill , Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District , Ryerson",43.657162,-79.378937


In [209]:
geo_merged.shape

(103, 5)